# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [22]:
from azureml.core import Workspace, Experiment
#Define a workspace
ws = Workspace.from_config()

#Create an experiment
exp = Experiment(workspace=ws, name="Hyperparameter_Tuning")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()


Workspace name: quick-starts-ws-243900
Azure region: westeurope
Subscription id: f5091c60-1c3c-430f-8d81-d802f6bf2414
Resource group: aml-quickstarts-243900


In [23]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProject"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 4, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-11-10T15:56:40.133000+00:00', 'errors': None, 'creationTime': '2023-11-10T14:30:43.587865+00:00', 'modifiedTime': '2023-11-10T14:30:53.493048+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [24]:
import pandas as pd
pd_path='https://raw.githubusercontent.com/AnnaDM87/Udacity_CAPSTONE/main/starter_file/Employee.csv'
df = pd.read_csv(pd_path)

df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [26]:
import os
os.listdir()

['.ipynb_checkpoints',
 'automl.ipynb',
 'conda_dependencies.yml',
 'hyperparameter_tuning.ipynb',
 'Logs',
 'outputs',
 'train.py',
 'training',
 'Users']

In [29]:
from train import clean_data
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
from azureml.data.dataset_factory import TabularDatasetFactory

#Use the clean_data function to clean your dataset.
x, y = clean_data(df)
x.head()

#Split the dataset into train and test dataset. Combine x_train and y_train. 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
df_train = pd.concat([x_train,y_train], axis=1)
df_test = pd.concat([x_test,y_test], axis=1)

#Convert x_train and y_train (Which are in pandas DataFrame format) to TabularDataset format.
try:
    os.makedirs('./data', exist_ok=True)
except OSError as error:
    print('New directory cannot be created')
    
path_train = 'data/train.csv'
path_test = 'data/test.csv'
df_train.to_csv(path_train)
df_test.to_csv(path_test)

datastore = ws.get_default_datastore()
datastore.upload(src_dir='data', target_path='data')

train_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/train.csv'))])
test_data = TabularDatasetFactory.from_delimited_files(path=[(datastore, ('data/test.csv'))])
print("Successfully converted the dataset to TabularDataset format.")

ModuleNotFoundError: No module named 'train'

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {}

# TODO: Put your automl config here

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model=run.get_output()
print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best accuracy run metrics ",best_run.get_metrics(name='accuracy'))
print("Best run details :", best_run.get_details())

In [ ]:
#TODO: Save the best model
best_run.get_file_names()

In [ ]:
automl_model = best_run.register_model(model_name = 'automl_model',
                                              model_path = 'outputs/model.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
